In [1]:
import pandas as pd
from tqdm import tqdm
import os

# CONNECT to REDSHIFT

In [2]:
from sqlalchemy import create_engine
import psycopg2
import dotenv

from os.path import expanduser
home = expanduser("~")

dotenv.load_dotenv(fr"{home}\creds\local_postgres.txt")

connection_params = dict(
        host=os.getenv('host')
        , dbname=os.getenv('dbname')
        , user=os.getenv('user')
        , password=os.getenv('password')
        , port=os.getenv('port')
)

week_schema = 'week7'

engine = create_engine(f'postgresql+psycopg2://{os.getenv("user")}:{os.getenv("password")}@{os.getenv("host")}:{os.getenv("port")}/{os.getenv("dbname")}')
engine.connect()

# Read CSVs into dataframes and upload

In [3]:
# walk through Tablas subdirectory and read all csv files into list of dataframes
df_dict = {}
for filename in tqdm(os.listdir('Tablas')):
    if filename.endswith('.csv'):
        pretty_filename = filename[:-4]
        df_dict[pretty_filename] = pd.read_csv('./Tablas/' + filename)
        df_dict[pretty_filename].to_sql(pretty_filename, engine, index=False, if_exists='replace', schema=week_schema)

100%|██████████| 12/12 [00:14<00:00,  1.24s/it]


In [4]:
df_dict.keys()

dict_keys(['countryregioncurrency', 'currencyrate', 'product', 'productcategory', 'productdescription', 'productmodelproductdescriptionculture', 'productreview', 'productsubcategory', 'salesorderdetail', 'salesorderheader', 'salesperson', 'salesterritory'])

In [5]:

stmt = f"""
SELECT * 
FROM week7.product p 
INNER JOIN week7.productreview pr
ON p.productid = pr.productid

"""

reviews = pd.read_sql_query(stmt, engine)


In [6]:
# which are the top salesmen
stmt = f"""
SELECT soh.salespersonid, SUM(soh.subtotal) AS total_sales
FROM week7.salesorderheader soh
INNER JOIN week7.salesperson sp
ON soh.salespersonid = sp.businessentityid

GROUP BY soh.salespersonid
ORDER BY total_sales DESC
"""


salesmen_sales = pd.read_sql_query(stmt, engine)
salesmen_sales.head(10)


,salespersonid,total_sales
0,276.0,1.036701e+07
1,277.0,1.006580e+07
2,275.0,9.293903e+06
3,289.0,8.503339e+06
4,279.0,7.171013e+06
5,281.0,6.427006e+06
6,282.0,5.926418e+06
7,290.0,4.509889e+06
8,283.0,3.729945e+06
9,278.0,3.609447e+06
